We'll start out with a plain and simple chat bot using OpenAI and gpt-4o-mini.  You will need an apikey for OpenAI to run this example.  
You can use a different model if you'd like. 

For this demo, in addition to an `OPENAI_API_KEY`, you will need a Mindlytics api key and project id.  These can be sent in environment variables; `MLSDK_API_KEY` and `MLSDK_PROJECT_ID`.  To run this notebook from the command line, you can

```sh
OPENAI_API_KEY=xxx MLSDK_API_KEY=yyy MLSDK_PROJECT_ID=zzz poetry run jupyter lab examples/langchain/chatbot.ipynb
```

Lets start out with the required imports and checks for environment variables.

In [ ]:
import os
import sys
import asyncio
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.callbacks import AsyncCallbackHandler

if not os.getenv("OPENAI_API_KEY"):
    print("Error: OPENAI_API_KEY environment variable not set.")
    print("Please set your OpenAI API key:")
    print("export OPENAI_API_KEY='your-api-key-here'")
    sys.exit(1)

if not (os.getenv("MLSDK_API_KEY") and os.getenv("MLSDK_PROJECT_ID") ):
    print("Error: Mindlytics environment is not set up.  The latter part of this demo will not work without them")


Langchain is part of the development dependencies for this repository, so you should have no trouble importing it.  We are going 
to make a streaming chatbot, so we will need a `AsyncCallbackHandler` with a `on_llm_new_token()` method which will print the response tokens
as they arrive from the LLM.

In [ ]:
class AsyncStreamingCallbackHandler(AsyncCallbackHandler):
    """Custom async callback handler for streaming output to stdout."""

    async def on_llm_new_token(self, token: str, **kwargs) -> None:
        """Called when a new token is generated."""
        print(token, end="", flush=True)


Now lets create a function to initialize the LLM.

In [ ]:
MODEL = "gpt-4o-mini"  # You can adjust this if you'd like

def initialize_llm():
    # Initialize the LLM with async streaming callback
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.7,
        streaming=True,
        # Add our callback handler to print tokens as they arrive
        callbacks=[AsyncStreamingCallbackHandler()], 
    )

    return llm


We need a small routine to get user input from the terminal.  We can do this by executing `input()` in a thread to avoid blocking the event loop.

In [ ]:
async def get_user_input(prompt: str) -> str:
    """Get user input asynchronously."""
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(None, input, prompt)


And now lets create a function that controls the actual chat loop.  This loop will get the next user input, add it to chat history, invoke the LLM 
with the user input, and save the LLM response in chat history.  It will also handle the exit conditions.

In [ ]:
async def chat_loop(llm):
    """Main async chat loop that prompts for input and streams responses."""
    print("Async LangChain CLI Chat Application")
    print("Type 'quit', 'exit', or press Ctrl+C to exit\n")

    message_history: List[BaseMessage] = []

    while True:
        try:
            # Get user input asynchronously
            user_input = await get_user_input("You: ")
            user_input = user_input.strip()

            # Check for exit commands
            if user_input.lower() in ["quit", "exit", "q"]:
                print("\nGoodbye!")
                break

            # Skip empty input
            if not user_input:
                continue

            # Create message and send to LLM
            message = HumanMessage(content=user_input)
            message_history.append(message)

            print("Assistant: ", end="", flush=True)

            # Stream the response asynchronously
            response = await llm.ainvoke(message_history)
            if hasattr(response, "content"):
                assistant_message = AIMessage(content=response.content)
                message_history.append(assistant_message)

            print("\n")  # Add newline after streaming response

        except KeyboardInterrupt:
            print("\n\nGoodbye!")
            break
        except Exception as e:
            print(f"\nError: {e}")
            print("Please try again.\n")


Finally, the main loop, which initializes the LLM and runs the chat loop.

In [ ]:
async def main():
    """Main async function to run the CLI application."""
    try:
        # Initialize the LLM
        llm = initialize_llm()

        # Start the async chat loop
        await chat_loop(llm)

    except Exception as e:
        print(f"Failed to initialize application: {e}")
        sys.exit(1)



In a normal script, you'd run the `main()` function like this:

```python
asyncio.run(main())
```

This is a notebook however, so instead of executing the `main()` function, lets step through some of it manually.

In [ ]:
# Initialize the LLM
llm = initialize_llm()

# Initialize chat history
message_history: List[BaseMessage] = []

# Create a user message
message = HumanMessage(content="I am looking for a good cup of coffee in Fremont CA.")
print(f"User: {message.content}")

# Add it to history
message_history.append(message)

# Get a response from the LLM
print("Assistant: ", end="", flush=True)
response = await llm.ainvoke(message_history)

# And add the final assistant text to message history
assistant_message = AIMessage(content=response.content)
message_history.append(assistant_message)

print("\n")  # Add newline after streaming response

You should see some assistant output above.  Lets do one more round, to make sure history is working.

In [ ]:
message = HumanMessage(content="Does Peet's have any donuts?")
print(f"User: {message.content}")
message_history.append(message)

print("Assistant: ", end="", flush=True)
response = await llm.ainvoke(message_history)
assistant_message = AIMessage(content=response.content)
message_history.append(assistant_message)
print("\n") 


Now we are going to add support for the `Mindlytics` service.  Lets add some imports. 

In [ ]:
# import Mindlytics API client and session, and MLEvent to capture analysis events.
from mlsdk import Client, Session, MLEvent

# import mindlytics llm callback handler for langchain
from mlsdk.helpers.langchain import MLChatRecorderCallback

# to get a unique device id for a Mindlytics session
import uuid

# This should give you a repeatable, unique id for your computer
mac = uuid.getnode()
device_id = f"{mac:012x}"


To the LLM initialization code, we are going to add another callback.  This callback is a helper available on the Mindlytics SDK which will
capture and send conversation turns to the Mindlytics service.  This callback handler requires an open Mindlytics "session" to be passed in its contructor, so 
our existing `initialize_llm()` needs to be modified slightly: 

In [ ]:
def initialize_llm(session: Session):
    # A Mindlytics session is passed in...
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.7,
        streaming=True,
        # Add this to enable token counts from OpenAI, which we can send to Mindlytics to capture costs
        model_kwargs={
            "stream_options": {"include_usage": True},
        },
        callbacks=[
            AsyncStreamingCallbackHandler(),
            MLChatRecorderCallback(session),  # Add the conversation recorder callback, passing session as an argument
        ],
    )

    return llm


The original `chat_loop()` function would not change in any way.  But `main()` would change to include the Mindlytics bits.

For this demo, we are going to use an optional feature of the Mindlytics service.  We are going to define callbacks to capture Mindlytics analytics events (and errors) from a websocket connection established and maintained within the Mindlytics sdk.  Most applications would probably skip this part and use the Mindlytics SaaS portal to view analytics, but in this demo we want to see what Mindlytics is doing in the background.  We are going to capture events asynchroniously and look at them when the conversation is complete.

In [ ]:
# Arrays to hold captured events and errors
ml_events: List[MLEvent] = []
async def on_event(event: MLEvent):
    # Handle Mindlytics events here
    ml_events.append(event)

ml_errors: List[Exception]  = []
async def on_error(error: Exception):
    # Handle Mindlytics errors here
    ml_errors.append(error)


Initialize the Mindlytics client.

In [ ]:
# The client constructor will read MLSDK_API_KEY and MLSDK_PROJECT_ID 
ml_client = Client()


Now we'll create a session context.  This requires either a "user id" or a "device id".  If you know the id of your user you'd pass that.  Otherwise you should pass a device-unique id.  See the Mindlytics documentation for a detailed explination of this parameter.  Since we wish to capture real time events, we'll pass our capture callbacks here as well.  If you do not pass capture callbacks, the websocket feature will not be activated.

In [ ]:
# Create a session context.
# Clear our event and error capture arrays (in case you execute this cell again!)
ml_events = []
ml_errors = []
session_context = ml_client.create_session(
    device_id=device_id, # created earlier
    on_event=on_event,   # capture events
    on_error=on_error,   # capture errors
)


We are going to use `session_context` in a python context.  As you know, this is a safe way to manage resources in python.  The Mindlytics sdk can take advantage of this to manage the communication with the Mindlytics service, and can automate some of the steps involved in conversation management.  You don't need to use the session in a context, but this is the easiest and safest way if you do not require more fine grain control over the Mindlytics session.

Since we already have a chat history, we're going to play it again on a new LLM instance.  Here's the code to do it:

In [ ]:
# Start the session
async with session_context as session:
    # Initialize the LLM with the open session
    llm = initialize_llm(session)
    # Play back our previous chat history to the LLM
    new_message_history: List[BaseMessage] = []
    for message in message_history:
        if isinstance(message, HumanMessage):
            new_message_history.append(message)
            print(f"User: {message.content}")
            print("Assistant: ", end="", flush=True)
            response = await llm.ainvoke(new_message_history)
            assistant_message = AIMessage(content=response.content)
            new_message_history.append(assistant_message)
            print('\n')

Once this context is done and before its is completely exited, the Mindlytics sdk will ensure all events have been sent to the Mindlytics service, and because we are using the websockets feature, the sdk waits until it sees a final "Session Ended" event to ensure all events have been captured.  You might see a small delay while the sdk is waiting for the final event.

In [ ]:
print(f"We received {len(ml_events)} events and {len(ml_errors)} errors.")


In [ ]:
if len(ml_errors) > 0:
    print()
    print("Mindlytics Errors:")
    for error in ml_errors:
        print("------------------------------------------------")
        print(error)


In [ ]:
for event in ml_events:
    print(event.event)

You should see some events.  You should see a "Session Started" at the beginning and a "Session Ended" at the end.  Within the session there should be a "Conversation Started"/"Conversation Ended" pair, and within the conversation you will see a number of events that were captured.  Every event contains a lot of detail.  Lets take a detailed look at the "Conversation Summary":

In [ ]:
summary = next((event for event in ml_events if event.event == 'Conversation Summary'), None)
if summary is None:
    print("Cannot find the Conversation Summary event!")

for key, value in summary.properties.items():
    print(f"{key}: {value}")


You can see some statistics about the conversation, the summary and the sentiment of the user.  Because we tracked token usage, we have a final cost calculation based on the `MODEL` you used to run the conversation.  

If you want to continue this demo by examining some of the other events in detail, here is a helper function to display one or more events by name:


In [ ]:
def show_events(event_name):
    for event in ml_events:
        if event.event == event_name:
            print()
            print(f"Event: {event.event}")
            print(f"  timestamp: {event.timestamp}")
            for prop in ["session_id", "conversation_id", "event_id", "origin_event_id"]:
                if hasattr(event, prop):
                    print(f"  {prop}: {getattr(event, prop)}")
            if hasattr(event, "properties"):
                print("  properties:")
                for key, value in event.properties.items():
                    print(f"    {key}: {value}")
            if hasattr(event, "user_traits"):
                print("  user_traits:")
                for key, value in event.user_traits.items():
                    print(f"    {key}: {value}")


And use it like this:

In [ ]:
show_events("Intent Informed")
show_events("Intent Closed")